# Colab × GitHub (шаблон с SSH-ключами)

Цель: работа с GitHub через SSH. Ключи храним в Google Drive, чтобы сохранялись между сессиями.

Алгоритм:
1. Смонтировать Drive.
2. Сгенерировать ключ (или использовать существующий) в `MyDrive/.ssh`.
3. Добавить публичный ключ в GitHub → Settings → SSH and GPG keys.
4. Клонировать репозиторий по SSH и работать как обычно.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import subprocess, pathlib
import datetime, pytz

moscow_tz = pytz.timezone("Europe/Moscow")

In [3]:
def sh(cmd, cwd=None, check=True):
    return subprocess.run(cmd, cwd=cwd, shell=True, text=True, capture_output=True, check=check)

def sh_live(cmd, cwd=None):
    """Выполнить команду и сразу печатать stdout/stderr в ячейку"""
    return subprocess.run(cmd, cwd=cwd, shell=True)

def git_config(user_name: str, user_email: str):
    sh(f'git config --global user.name "{user_name}"')
    sh(f'git config --global user.email "{user_email}"')
    sh('git config --global credential.helper ""')

git_config("SIA86", "sfron4ik@gmail.com")

In [4]:
from pathlib import Path

DRIVE_SSH_DIR = Path('/content/drive/MyDrive/.ssh')
DRIVE_SSH_DIR.mkdir(parents=True, exist_ok=True)
ID_KEY = DRIVE_SSH_DIR / 'id_ed25519'
ID_PUB = DRIVE_SSH_DIR / 'id_ed25519.pub'

if not ID_KEY.exists():
    sh(f'ssh-keygen -t ed25519 -N "" -f "{ID_KEY}"')
    print("Сгенерирован новый ключ.")
else:
    print("Найден существующий ключ.")

sh('mkdir -p ~/.ssh && chmod 700 ~/.ssh')
sh(f'ln -sf "{ID_KEY}" ~/.ssh/id_ed25519')
sh(f'ln -sf "{ID_PUB}" ~/.ssh/id_ed25519.pub')
sh('chmod 600 ~/.ssh/id_ed25519 && chmod 644 ~/.ssh/id_ed25519.pub')
sh('ssh-keyscan github.com >> ~/.ssh/known_hosts || true')
sh('chmod 644 ~/.ssh/known_hosts')

print("Публичный ключ для GitHub:")
print(ID_PUB.read_text())

Найден существующий ключ.
Публичный ключ для GitHub:
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKot/XiGZ9/rvjmmZsCuGn1O0CIQzGUeiYLjyWCpdWN8 root@01bff0625f36



In [5]:
res = sh('ssh -o StrictHostKeyChecking=yes -T git@github.com', check=False)
print(res.stdout or res.stderr)

Hi SIA86! You've successfully authenticated, but GitHub does not provide shell access.



In [6]:
OWNER_AND_REPO = "SIA86/pretrained_rl_ppo_model"  # например: octocat/Hello-World
DEST_DIR = "/content/drive/MyDrive/stocks/pretrained_rl_ppo"
ssh_url = f"git@github.com:{OWNER_AND_REPO}.git"

%cd "$DEST_DIR"


/content/drive/MyDrive/stocks/pretrained_rl_ppo


In [ ]:
# Клонировать удаленный репо
!git clone ssh_url DEST_DIR

In [ ]:
# Посмотреть ветки
!git branch -a

/content/drive/MyDrive/stocks/pretrained_rl_ppo
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/main


In [ ]:
# Подтянуть изменения
!git pull

/content/drive/MyDrive/stocks/pretrained_rl_ppo
Already up to date.


In [ ]:
# Переключиться или создать ветку
!git checkout -b main

/content/drive/MyDrive/stocks/pretrained_rl_ppo
fatal: A branch named 'main' already exists.


In [10]:
# проверка статуса
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   colab_github_ssh.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [11]:
# Добавить все изменения разом
now = datetime.datetime.now(moscow_tz).strftime("%d-%m-%y %H:%M")
!git add --all
!git commit -m f"Обновление {now}"
!git push -u origin main

error: read error while indexing colab_github_ssh.ipynb: Operation not permitted
error: colab_github_ssh.ipynb: failed to insert into database
error: unable to index file 'colab_github_ssh.ipynb'
fatal: updating files failed
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   colab_github_ssh.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
Branch 'main' set up to track remote branch 'main' from 'origin'.
Everything up-to-date


In [9]:
# Добавить изменения по конкретному файлу
!git add pytest.ipynb
!git commit -m f"Добавлены pytests для q_label_matching"
!git push -u origin main

[main b3fa504] fДобавлены pytests для q_label_matching
 1 file changed, 1 insertion(+)
 create mode 100644 pytest.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 3.99 KiB | 314.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:SIA86/pretrained_rl_ppo_model.git
   7287af3..b3fa504  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
